#Продвинутый Python, ДЗ-4

Правила игры:

В домашке 6 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно рисовать графики с помощью plotly или же Dash. Обратите внимание, что все графики и оси должны быть подписаны, графики должны выглядеть аккуратно (за это будем штрафовать)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [25]:
!pip install wget


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

# Часть 1. Plotly

В данной части для заданий нужно использовать Plotly для того, чтобы отобразить графики

## Задание 1 (15 баллов)

Отобразите на карте всех продавцов (в качестве точек - координаты, в качестве размера и цвета точек - число продавцов). При наведении точек должно отображаться название штата, города, а также число продавцов

*Hint: внимательно посмотрите на датасет locations. Если делать задачу максимально втупую, то у вас либо крашнется/зависнет отрисовка, либо на это нельзя будет нормально смотреть*

In [39]:
import unicodedata

def normalize_city_name(city_name):
    city_name = city_name.replace('-', '')
    city_name = city_name.replace('*', '')
    city_name = city_name.replace('º', 'o')
    city_name = city_name.replace('.', '')
    city_name = city_name.replace('´', '')
    city_name = city_name.replace(' ', '')
    city_name = ''.join(
        c for c in unicodedata.normalize('NFD', city_name)
        if unicodedata.category(c) != 'Mn'
    )
    return city_name

grouped_sellers=sellers.groupby('seller_city').agg({'seller_id':'size'}).reset_index()
new_locations=pd.merge(location, grouped_sellers,right_on = 'seller_city', left_on='geolocation_city', how='left').rename(columns={'seller_id':'amount'})
new_locations['city']=new_locations['geolocation_city'].apply(normalize_city_name)
new_locations=new_locations.groupby('city').agg({'geolocation_lat':'mean','geolocation_lng':'mean','geolocation_state':'first','amount':'first'}).reset_index()
df=new_locations.dropna(subset=['amount']).sort_values('amount',  ascending=False).reset_index()
df['text'] = df['city'] + ', '+df['geolocation_state']+'<br>Продавцов: ' + (df['amount']).astype(str)+'.'

In [234]:
import plotly.graph_objects as go
import pandas as pd

limits = [(0,5),(6,20),(21,50),(51,100),(101,546)]
colors = ["#8B0001","#B12E21","#D65D42","#E97452","lightgrey"]
cities = []
scale = 5000

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = df_sub['geolocation_lng'],
        lat = df_sub['geolocation_lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['amount'],
            color = colors[i],
            line_color='rgb(0, 0, 0)',
            line_width=1,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))

fig.update_layout(
        title_text = 'Количество продавцов по городам Бразилии',
        showlegend = True,
        geo = dict(
            scope = 'south america',
            landcolor = 'rgb(222, 184, 135)',
        )
    )

fig.show()

In [52]:
import plotly.express as px

fig = px.scatter_mapbox(df, lat="geolocation_lat", lon="geolocation_lng", color="amount", size="amount",
                  color_continuous_scale=px.colors.sequential.Viridis, size_max=25, zoom=3,
                  mapbox_style="carto-positron", text='text',title = 'Количество продавцов по городам Бразилии')
fig.show()

## Задание 2 (15 баллов)

Отобразите на графике число заказов по дням с разбивкой по статусам. Также создайте график со средней оценкой отзыва (аггрегируйте по дням). Для обоих графиков должен быть слайдер по датам

In [139]:
new_df=pd.merge(orders, reviews, on='order_id', how='inner')
new_df['order_purchase_date'] = pd.to_datetime(new_df['order_purchase_timestamp']).dt.date
grouped =new_df.groupby(['order_purchase_date', 'order_status']).size().reset_index(name='counts')
grouped2=new_df.groupby('order_purchase_date').agg({'review_score':'mean'}).reset_index()

In [148]:
fig = go.Figure()
statuses = grouped['order_status'].unique()
colors=['#FBECDB','#F3CBBD','#8DA47E','#AF8C72','#938F43','#DBBC8E','#C6AC85','#874741']
for index, status in enumerate(statuses):
    color = colors[index]
    fig.add_trace(go.Scatter(
        x=grouped[grouped['order_status'] == status]['order_purchase_date'],
        y=grouped[grouped['order_status'] == status]['counts'],
        name=status,
        line=dict(color=color) 
    ))


fig.update_layout(
    title='Число заказов по дням с разбивкой по статусам',
    dragmode="pan",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1d', step='day', stepmode='backward'),
                dict(count=7, label='1w', step='day', stepmode='backward'),
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

fig.show()


In [150]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=grouped2['order_purchase_date'],
    y=grouped2['review_score'],
    name='Оценка',
    line=dict(color='#874741')
))
fig.update_layout(
    title='Средняя оценка отзыва по дням',
    dragmode="pan",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100),
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1d', step='day', stepmode='backward'),
                dict(count=7, label='1w', step='day', stepmode='backward'),
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)

fig.show()

## Задание 3 (20 баллов)

Выберите топ-5 продавцов с более чем 20 продажами, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

In [20]:
import unicodedata

def normalize_city_name(city_name):
    city_name = city_name.replace('-', '')
    city_name = city_name.replace('*', '')
    city_name = city_name.replace('º', 'o')
    city_name = city_name.replace('.', '')
    city_name = city_name.replace('´', '')
    city_name = city_name.replace(' ', '')
    city_name = ''.join(
        c for c in unicodedata.normalize('NFD', city_name)
        if unicodedata.category(c) != 'Mn'
    )
    return city_name

new_locations=location.copy()
new_locations['city']=new_locations['geolocation_city'].apply(normalize_city_name)
new_locations=new_locations.groupby('city').agg({'geolocation_lat':'mean', 'geolocation_lng':'mean', 'geolocation_state':'first'}).reset_index()
new_customers=customers.copy()
new_customers['customer_city']=new_customers['customer_city'].apply(normalize_city_name)
new_sellers=sellers.copy()
new_sellers['seller_city']=new_sellers['seller_city'].apply(normalize_city_name)

sells=pd.merge(new_sellers, items, on='seller_id', how='inner')
sells=pd.merge(sells, new_locations, left_on='seller_city',right_on='city', how='inner').rename(columns={'geolocation_lat':'geolocation_lat_seller','geolocation_lng':'geolocation_lng_seller'})
sells=pd.merge(sells, orders, on='order_id', how='inner')
sells=pd.merge(sells, new_customers, on='customer_id', how='inner')
sells=pd.merge(sells, new_locations, left_on='customer_city',right_on='city', how='inner').rename(columns={'geolocation_lat':'geolocation_lat_customer','geolocation_lng':'geolocation_lng_customer'})
sells=pd.merge(sells, products, on='product_id', how='inner')	
sells=pd.merge(sells, translation, on='product_category_name', how='inner')
sells=sells.drop_duplicates(subset=['order_id'])
sells['foreign']=sells.apply(lambda x: x['seller_state']!=x['customer_state'])
sells_grouped=sells.groupby('seller_id').agg({'foreign':'mean','order_id':'size','seller_city':'first','customer_city':lambda x: list(x),'geolocation_lat_seller':'first','geolocation_lng_seller':'first','geolocation_lat_customer':lambda x:list(x),'geolocation_lng_customer':lambda x:list(x)}).rename(columns={'order_id':'amount'})
sells_grouped=sells_grouped[sells_grouped['amount']>20].sort_values(['foreign','amount'], ascending=False)


In [126]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

colors=['#DFFF00','#FFBF00','#FF7F50','#DE3163','#40E0D0']
for i in range(5):
    for seller in range(len(sells_grouped['geolocation_lng_customer'][i])):
        fig.add_trace(
            go.Scattergeo(
                locationmode = 'country names',
                lon = [sells_grouped['geolocation_lng_seller'][i], sells_grouped['geolocation_lng_customer'][i][seller]],
                lat = [sells_grouped['geolocation_lat_seller'][i], sells_grouped['geolocation_lat_customer'][i][seller]],
                mode = 'lines+markers',
                hoverinfo = 'text',  
                hovertext = sells_grouped['customer_city'][i][seller],
                line = dict(width = 1,color = colors[i]),
                opacity = 0.4,
                marker = dict(
                size = 4,
                color = colors[i],
                ),
            )
        )
for i in range(5):
    fig.add_trace(go.Scattergeo(
        lon = [sells_grouped['geolocation_lng_seller'][i]],
        lat = [sells_grouped['geolocation_lat_seller'][i]],
        hoverinfo = 'text',
        name=sells_grouped['seller_city'][i],
        text = str(sells_grouped['seller_city'][i]),
        mode = 'markers',
        opacity=0.5,
        textfont = {"color": 'black',
                    "family":'Times New Roman',
                    "size":10},
        marker = dict(
            size = 3,
            color = colors[i],
            ),
    ))
for i in range(5):
    fig.add_trace(go.Scattergeo(
        lon = [sells_grouped['geolocation_lng_seller'][i]],
        lat = [sells_grouped['geolocation_lat_seller'][i]],
        text = str(sells_grouped['seller_city'][i]),
        mode = 'text',
        textfont = {"color": 'black',
                    "family":'Arial Black',
                    "size":7},
    ))

fig.update_layout(
    title_text = 'Отправки продавцов, которые чаще всего отправляют посылки в другие штаты',
    showlegend = False,
    geo = dict(
        scope = 'south america',
        projection_type = 'natural earth',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
)

fig.show()

# Часть 2. Dash

В данной части нужно сделать и сдать один .py файл (можно назвать dash_task.py), который будет разворачивать страничку с нарисованными графиками (с помощью Dash)

## Задание 4 (20 баллов)

Сделайте сайт, где будет отображаться следующее:

* График, показывающий распределение по категориям продаж (то есть по тому, что продают селлеры) в штате

* График, показывающий распределение по категориям покупок (то есть по тому, что покупают пользователи) в штате

* Фильтр по штатам (должна быть возможность как выбрать отдельный штат, так и вся Бразилия в целом)

* Возможность выбрать, какие статусы продаж учитывать (чтобы можно было выбрать несколько статусов)

* Слайдер по датам (чтобы можно было выбрать даты, по которым можем смотреть результаты)


In [133]:
sells=pd.merge(sellers, items, on='seller_id', how='inner')
sells=pd.merge(sells, orders, on='order_id', how='inner')
sells=pd.merge(sells, customers, on='customer_id', how='inner')
sells=pd.merge(sells, products, on='product_id', how='inner')	
sells=pd.merge(sells, translation, on='product_category_name', how='inner')

In [135]:
sells=sells[['seller_state','price','customer_state','product_category_name_english','order_purchase_timestamp','order_status']]

In [136]:
sells.to_csv('/Users/aliya/sells.csv')

## Задание 5 (15 баллов)

Добавить карту, на которой можно будет посмотреть на количество продавцов или на количество покупателей (на него также должны работать фильтры по датам и по статусам, реализованные в прошлом занятии). Аггрегацию нужно делать по штатам. Чтобы вам помочь, вот snippet кода, который можно использовать для отображения:

In [138]:
import json
import pandas as pd
import plotly as plt
import plotly.express as px
from urllib.request import urlopen

# Brazil coordinates / shape
with urlopen(
    "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"
) as response:
    brazil = json.load(response)

# Since the database doesn't have an ID or feature using which values will be mapped between the coordinate/shape database and soybean database, we are adding an ID ourselves.
for feature in brazil["features"]:
    feature["id"] = feature["properties"]["sigla"]

# Здесь в качестве id, который будет отображаться, указано сокращение названия штата (как и в наших данных)

*Hint: обратите внимание, что покупатели и продавцы могут быть не во всех штатах, а это значит, что они могут не отображаться на карте. Эту ситуацию нужно обрабатывать и поправить*

## Задание 6 (15 баллов)

Сделайте карту кликабельной: по клику на штат должен показываться сам штат (и только он), все остальные графики обновляются по выбранному штату